In [1]:
# it's possible to recover and change decimal numbers context
# for example to change rounding / precision settings

import decimal
from decimal import Decimal


decimal.getcontext()  # this is a global context

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

In [2]:
decimal.getcontext().prec = 14
decimal.getcontext()

Context(prec=14, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

In [3]:
decimal.getcontext().prec = 28
decimal.getcontext()

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

In [4]:
old_prec = decimal.getcontext().prec
decimal.getcontext().prec = 4
print(Decimal(Decimal(1) / Decimal(3)))

0.3333


In [5]:
decimal.getcontext().prec = old_prec
print(Decimal(Decimal(1) / Decimal(3)))

0.3333333333333333333333333333


In [6]:
class precision:
    def __init__(self, prec):
        self.prec = prec
        self.current_prec = decimal.getcontext().prec

    def __enter__(self):
        print("__enter__", "instance ID:", id(self))
        decimal.getcontext().prec = self.prec

    def __exit__(self, exc_type, exc_value, exc_tb):
        decimal.getcontext().prec = self.current_prec
        return False


In [7]:
with precision(3):
    print(Decimal(Decimal(1) / Decimal(3)))

__enter__ instance ID: 4527271920
0.333


In [8]:
print(Decimal(Decimal(1) / Decimal(3)))
with precision(3):  # new context manager instance every time
    print(Decimal(Decimal(1) / Decimal(3)))
    with precision(5):
        print(Decimal(Decimal(1) / Decimal(3)))
        with precision(1):
            print(Decimal(Decimal(1) / Decimal(3)))
        print(Decimal(Decimal(1) / Decimal(3)))
    print(Decimal(Decimal(1) / Decimal(3)))
print(Decimal(Decimal(1) / Decimal(3)))

0.3333333333333333333333333333
__enter__ instance ID: 4527270144
0.333
__enter__ instance ID: 4527455264
0.33333
__enter__ instance ID: 4527607248
0.3
0.33333
0.333
0.3333333333333333333333333333


In [9]:
# decimal has it's own context manager

with decimal.localcontext() as ctx1:
    ctx1.prec = 3
    print(Decimal(Decimal(1) / Decimal(3)))
    with decimal.localcontext() as ctx2:
        ctx2.prec = 6
        print(Decimal(Decimal(1) / Decimal(3)))
print(Decimal(Decimal(1) / Decimal(3)))

0.333
0.333333
0.3333333333333333333333333333


#### context managers can be used for timing things

In [10]:
from time import perf_counter, sleep


class Timer:
    def __init__(self):
        self.elapsed = 0
        self.start = None
        self.stop = None

    def __enter__(self):
        self.start = perf_counter()
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        self.stop = perf_counter()
        self.elapsed = self.stop - self.start
        return False
   

In [11]:
with Timer() as t1:
    sleep(0.02)
    with Timer() as t2:
        sleep(0.01)
        with Timer() as t3:
            sleep(0.033)
        print("T3", t3.elapsed)
    print("T2", t2.elapsed)
print("T1", t1.elapsed)

T3 0.03851009900245117
T2 0.051063437000266276
T1 0.07606779499474214


#### Redirecting standard out

In [12]:
import sys


class OutToFile:
    DEFAULT_FILE_STDOUT = "std-context-manager.txt"

    def __init__(self, file_name: str = ""):
        self._file_name = file_name or self.DEFAULT_FILE_STDOUT
        self._file = None
        self._current_stdout = sys.stdout

    def __enter__(self):
        self._file = open(self._file_name, "w")
        sys.stdout = self._file

    def __exit__(self, exc_type, exc_value, exc_tb):
        sys.stdout = self._current_stdout
        self._file.close()
        return False


In [13]:
with OutToFile():
    print(sys.stdout)
    print("Line 1")
    print("Line 2")


In [14]:
with open(OutToFile.DEFAULT_FILE_STDOUT) as f:
    print(next(f), end="")
    print(next(f), end="")
    print(next(f), end="")

<_io.TextIOWrapper name='std-context-manager.txt' mode='w' encoding='UTF-8'>
Line 1
Line 2


#### Inject HTML tags around prints

In [15]:
class Tag:
    def __init__(self, tag_name: str):
        self._tag = tag_name

    def __enter__(self):
        print(f"<{self._tag}>", end="")

    def __exit__(self, exc_type, exc_value, exc_tb):
        print(f"</{self._tag}>")
        return False

In [16]:
with Tag("div"):
    print("test")
    with Tag("ul"):
        print("My list")
        with Tag("li"):
            print("el 1", end="")
        with Tag("li"):
            print("el 2", end="")
        with Tag("li"):
            print("el 3")

<div>test
<ul>My list
<li>el 1</li>
<li>el 2</li>
<li>el 3
</li>
</ul>
</div>


In [17]:
#### Using same context manager instance multiple times - identation levels


class ListMaker:
    def __init__(self, title: str, prefix: str = "- ", indent: int = 3):
        self._title = title
        self._prefix = prefix
        self._indent = indent
        self._current_indantation = 0
        print(title)

    def __enter__(self):
        self._current_indantation += self._indent
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        self._current_indantation -= self._indent
        return False

    def print(self, arg):
        print(" " * self._current_indantation + self._prefix + str(arg))


In [18]:
with ListMaker("My cool list") as lm:
    lm.print("item 1")
    lm.print("item 2")
    with lm as lm:
        lm.print("subitem 2.1")
        lm.print("subitem 2.2")
        lm.print("subitem 2.3")
        with lm as lm:
            lm.print("subitem 2.3.1")
            lm.print("subitem 2.3.2")
        lm.print("subitem 2.4")
    lm.print("item 3")

My cool list
   - item 1
   - item 2
      - subitem 2.1
      - subitem 2.2
      - subitem 2.3
         - subitem 2.3.1
         - subitem 2.3.2
      - subitem 2.4
   - item 3
